In [1]:
import csv
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding, Bidirectional,Conv1D,MaxPool1D,MaxPooling1D,GlobalMaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from mlxtend.plotting import plot_confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split


import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
from nltk.stem import PorterStemmer
import re
nltk.download('punkt')

!pip install prettytable
from prettytable import PrettyTable


import time

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BRAVO15\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BRAVO15\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#Converting Dataset to Dataframe :
articles = []
labels = []

with open("bbc-text.csv", 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[0])
        article = row[1]
        for word in STOPWORDS:
            token = ' ' + word + ' '
            article = article.replace(token, ' ')
            article = article.replace(' ', ' ')
        articles.append(article)
print(len(labels))
print(len(articles))

2225
2225


In [3]:
raw_df=pd.DataFrame({"Text":articles,"Labels":labels})
raw_df

,Text,Labels
0,tv future hands viewers home theatre systems ...,tech
1,worldcom boss left books alone former worldc...,business
2,tigers wary farrell gamble leicester say rus...,sport
3,yeading face newcastle fa cup premiership side...,sport
4,ocean twelve raids box office ocean twelve cr...,entertainment
...,...,...
2220,cars pull us retail figures us retail sales fe...,business
2221,kilroy unveils immigration policy ex-chatshow ...,politics
2222,rem announce new glasgow concert us band rem a...,entertainment
2223,how political squabbles snowball become common...,politics


In [4]:
#Inspecting our Dataframe :
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    2225 non-null   object
 1   Labels  2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


In [5]:
#Getting Number of Unique Classes in our dataset :
raw_df.Labels.unique(),print("\n Total number of Unique Target Classes : ",raw_df.Labels.nunique())


 Total number of Unique Target Classes :  5


(array(['tech', 'business', 'sport', 'entertainment', 'politics'],
       dtype=object),
 None)

In [6]:
# Text cleanup :
stemmer = PorterStemmer()

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
REMOVE_NUM = re.compile('[\d+]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
    text: a string
    return: modified initial string
    """
    # lowercase text
    text = text.lower() 

    # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    
    # Remove the XXXX values
    text = text.replace('x', '') 
    
    # Remove white space
    text = REMOVE_NUM.sub('', text)

    #  delete symbols which are in BAD_SYMBOLS_RE from text
    text = BAD_SYMBOLS_RE.sub('', text) 

    # delete stopwords from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) 
    
    # removes any words composed of less than 2 or more than 21 letters
    text = ' '.join(word for word in text.split() if (len(word) >= 2 and len(word) <= 21))

    # Stemming the words
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    
    return text

In [7]:
dataset=raw_df

In [8]:
dataset['Text']=dataset['Text'].apply(clean_text)
dataset['Text']

0       tv futur hand viewer home theatr system plasma...
1       worldcom boss left book alon former worldcom b...
2       tiger wari farrel gambl leicest say rush make ...
3       yead face newcastl fa cup premiership side new...
4       ocean twelv raid bo offic ocean twelv crime ca...
                              ...                        
2220    car pull us retail figur us retail sale fell j...
2221    kilroy unveil immigr polici echatshow host rob...
2222    rem announc new glasgow concert us band rem an...
2223    polit squabbl snowbal becom commonplac argu bl...
2224    souness delight euro progress boss graem soune...
Name: Text, Length: 2225, dtype: object

In [9]:
#Splitting raw data for Training and Testing :
text = dataset["Text"].values
labels = dataset['Labels'].values

X_train, y_train, X_test, y_test = train_test_split(text,labels, test_size = 0.20, random_state = 42)
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)


(1780,) (1780,)
(445,) (445,)


In [10]:
#Reshaping Labels Input :
X_test=X_test.reshape(-1,1)
y_test=y_test.reshape(-1,1)

print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)


(1780,) (1780, 1)
(445,) (445, 1)


In [11]:
#Let's Fix Some Common Parameters :
# The maximum number of words to be used. (most frequent)
vocab_size = 50000

# Dimension of the dense embedding.
embedding_dim = 128

# Max number of words in each complaint.
max_length = 200

# Truncate and padding options
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

In [12]:
# Tokenising , Converting Text To Sequences and Padding Our Data : 

tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
word_index

Found 19057 unique tokens.


{'<OOV>': 1,
 'said': 2,
 'mr': 3,
 'year': 4,
 'would': 5,
 'also': 6,
 'new': 7,
 'peopl': 8,
 'us': 9,
 'one': 10,
 'game': 11,
 'say': 12,
 'use': 13,
 'could': 14,
 'time': 15,
 'last': 16,
 'make': 17,
 'first': 18,
 'net': 19,
 'go': 20,
 'govern': 21,
 'like': 22,
 'two': 23,
 'play': 24,
 'take': 25,
 'world': 26,
 'get': 27,
 'compani': 28,
 'film': 29,
 'work': 30,
 'uk': 31,
 'show': 32,
 'firm': 33,
 'music': 34,
 'back': 35,
 'bn': 36,
 'want': 37,
 'best': 38,
 'told': 39,
 'market': 40,
 'win': 41,
 'plan': 42,
 'made': 43,
 'includ': 44,
 'month': 45,
 'report': 46,
 'servic': 47,
 'set': 48,
 'come': 49,
 'number': 50,
 'ad': 51,
 'way': 52,
 'player': 53,
 'week': 54,
 'three': 55,
 'countri': 56,
 'need': 57,
 'mani': 58,
 'parti': 59,
 'bbc': 60,
 'labour': 61,
 'look': 62,
 'epect': 63,
 'home': 64,
 'elect': 65,
 'may': 66,
 'nation': 67,
 'sale': 68,
 'good': 69,
 'help': 70,
 'day': 71,
 'well': 72,
 'call': 73,
 'minist': 74,
 'technolog': 75,
 'million': 76,


In [13]:
# Converting into Text to sequences and padding :
train_seq = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)

validation_seq = tokenizer.texts_to_sequences(y_train)
validation_padded = pad_sequences(validation_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)


In [14]:
#Just an Example to see the raw sentance , sentance in sequences, sentance in sequences with padding:
X_train[3],train_seq[3],train_padded[3]

('school tribut tv host carson peopl turn sunday pay tribut late us tv present johnni carson nebraska town grew carson host tonight show year die januari respiratori diseas emphysema live norfolk nebraska age eight join navi return regularli donat local caus old school friend among crowd school johnni carson theater carson one bestlov tv person us ask public memori lo angel live later life began showbusi career norfolk perform magic name great carsoni age donat includ norfolk high school build new perform art centr carson die presid bush led public tribut say present profound influenc american life entertain',
 [406,
  1949,
  118,
  533,
  4422,
  8,
  256,
  400,
  263,
  1949,
  671,
  9,
  118,
  414,
  2424,
  4422,
  7885,
  2002,
  1396,
  4422,
  533,
  5529,
  32,
  4,
  788,
  314,
  12214,
  1915,
  12215,
  210,
  4423,
  7885,
  480,
  688,
  534,
  7886,
  188,
  2140,
  1054,
  395,
  613,
  689,
  406,
  879,
  429,
  1103,
  406,
  2424,
  4422,
  9366,
  4422,
  10,
 

In [15]:
print('Shape of data tensor:', train_padded.shape)
print('Shape of data tensor:', validation_padded.shape)

Shape of data tensor: (1780, 200)
Shape of data tensor: (445, 200)


In [16]:
#Using One Hot Enocder to Enocde our Multi class Labels  :
encode = OneHotEncoder()

training_labels = encode.fit_transform(X_test)
validation_labels = encode.transform(y_test)

In [17]:
from tensorflow import keras
model = keras.models.load_model('model_english.h5')

In [18]:
# Testing Random Text and Predicted Classes :
import glob
for file in glob.glob('./News/*.txt'):
    f = open(file, 'r', encoding ='utf8')
    text = f.read()
    new_text = [clean_text(text)]
    seq = tokenizer.texts_to_sequences(new_text)
    padded = pad_sequences(seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    pred = model.predict(padded)
    acc = np.argmax(padded,axis=1)
    predicted_label = encode.inverse_transform(pred)
    #print(f'Product category id: {np.argmax(pred[0])}')
    a = (f'{file} predicted label is: {predicted_label[0]}\n')
    result = open('result.txt', 'a')
    result.write(a)
    with open('./result.csv','a') as f:
        writer = csv.writer(f)
        writer.writerow([file,str(predicted_label[0])])

   
    #print(f'Accuracy score: { acc.max() * 100}')



In [23]:
# Testing Random Text and Predicted Classes :
f = open('news.txt', 'r')
original_text=raw_df['Text']
text = f.read()
new_text = [clean_text(text)]
print(new_text)
seq = tokenizer.texts_to_sequences(new_text)
padded = pad_sequences(seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)
pred = model.predict(padded)
acc = np.argmax(padded,axis=1)
predicted_label = encode.inverse_transform(pred)
print('')
print(f'Predicted label is: {predicted_label[0]}')


['bblack friday tech retail fear suppli chain delay hit stock bbc newsbbc homepageskip contentaccess helpyour menubbc artshealthworld news tvin picturesr checknewsbeatlong readsbusinessmarket datanew economynew tech businessbusi sportglob car industryblack friday tech retail fear suppli chain delay hit stockpublish hour agosharecloseshar pagecopi linkabout sharingimag sourc getti imagestechnolog retail warn may insuffici stock meet black friday demand due delay suppli chainth imrg uk onlin retail associ said industri seen delay stock arrivingth real pinch point asia andi mulcahi imrg insight director saidshortag driver warehous staff send purchas good also worri busi accord mr mulcahyblack friday less week away annual event retail slash price entic shopper ahead christma periodit common retail buy high number unit specif black friday sometim even month advanc shortag hit countri around worldtoy shop warn christma shortag amid delayshowev delay four si week retail may find need chang pr